In [1]:
import os
import pandas as pd
import requests as req
import json
import numpy as np
import re
from unidecode import unidecode
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

MAX_NAME_SIZE = 50
DICT_SIZE = 26

Using TensorFlow backend.


## Getting the data

### From the API (sometimes down)

In [2]:
DATA_URL = 'https://api.got.show/api/characters/'

data = req.get(DATA_URL).content

### Reading from the downloaded data

In [4]:
# OPTIONAL

with open(os.path.join('data', 'characters.json'), 'r') as f:
    data = f.read()

In [5]:
df = pd.read_json(data, orient='records')
len(df)

2028

### Getting only some columns

The prediction will be name based, so the name of the character is a important data. To know the age of the character we will get the columns `dateOfBirth` and the `dateOfDeath`, with the years of birth and death, respectively.

In [6]:
df = df[['name', 'dateOfBirth', 'dateOfDeath']]

## Preprocessing the data

### Processing the name

First tratament that we will do in the name is removing some special words, like words accentuation. For example, `Í` will be replaced by `I`, accentuations are unnecessary informations.

The second processing is remove characters that are not words, like symbols and spaces, that also are unnecessary informations.

Finally, let the name in lowercase.

### Processing the age

In the dataset we will have 3 cases of ages: 

- a) don't has the `dateOfBith`
- b) don't has the `dateOfDeath`
- c) has both

The cases a) are bad there no way how to calculare the age of that character, we will discart them.

In the cases b) the characters still alive, so we will define the age as -1, this value will represent that the character is alive (lucky).

The best case, or case c), we will calcule the age of this character by the difference between `dateOfBirth` and `dateOfDeath`.

In [7]:
def preprocess_name(name):
    
    # decode unicode
    out_name = unidecode(name.lower())
    
    # getting only words
    out_name = re.sub(r'\W', '', out_name)
    
    # truncate the name
    if len(out_name) > MAX_NAME_SIZE:
        out_name = out_name[:MAX_NAME_SIZE]
    
    return out_name

def preprocess_age(birth, death):
    if np.isnan(death):
        # still alive
        return -1
    else:
        # is dead
        return int(death - birth)

In [8]:
# getting only the chatacters with a date of birth

df_only_born = df.dropna(subset=['dateOfBirth'])
len(df_only_born)

478

In [9]:
# getting all the names and applying the preprocessing algorithm

df_names = df_only_born['name'].apply(preprocess_name)

In [10]:
# computing the ages

list_ages = []

for birth, death in zip(list(df_only_born['dateOfBirth']), list(df_only_born['dateOfDeath'])):
    list_ages.append(preprocess_age(birth, death))

## Encoding

In the step we will prepare the data as the input of our neural model.

### Encoding the names

We will encode the names replacing the letters by numbers, each letter will be represented by the same number in all names. In that step we will also pad the name, that is, let them with the same size.

### Encoding the ages

The range between the lowest age and the greattest is too big, that is bad to the neural machine. So, we will scale the values to variate between 0 and 1, this is better to the machine predict a number. Later we can use the same object to apply the inverse transform in the predicted value.

In [11]:
def encode_name(name):
    return one_hot(' '.join(list(name)), DICT_SIZE)

In [12]:
encoded_names = []

# encode names in one hot
for index, name in enumerate(list(df_names)):
    encoded_names.append(encode_name(name))

# pad sequences
encoded_names = pad_sequences(encoded_names, maxlen=MAX_NAME_SIZE, padding='post')
encoded_names = np.array(encoded_names)
encoded_names.shape

(478, 50)

In [13]:
# scaling the ages

mm_scaler = MinMaxScaler()
scaler = mm_scaler.fit([[age] for age in list_ages])

In [14]:
ages_scaled = scaler.transform([[age] for age in list_ages])
ages_scaled = np.array(ages_scaled)
ages_scaled.shape

(478, 1)

# Neural Machine

That if the fun part, make a model to get a name and predict which is the age of death ot that person

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten

model = Sequential()
model.add(Embedding(DICT_SIZE, 50, input_length=MAX_NAME_SIZE))
model.add(Flatten())
model.add(Dense(units=25, activation='sigmoid'))
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            1300      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                62525     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 64,096
Trainable params: 64,096
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer="adam", loss='binary_crossentropy',  metrics=['accuracy'])

history = model.fit(x=encoded_names, y=ages_scaled, epochs=100, verbose=1, shuffle=True)

Epoch 1/100
478/478 [==============================] - 0s 408us/step - loss: 0.2637 - acc: 0.6590
Epoch 2/100
478/478 [==============================] - 0s 218us/step - loss: 0.2623 - acc: 0.6590
Epoch 3/100
478/478 [==============================] - 0s 163us/step - loss: 0.2614 - acc: 0.6590
Epoch 4/100
478/478 [==============================] - 0s 193us/step - loss: 0.2606 - acc: 0.6590
Epoch 5/100
478/478 [==============================] - 0s 154us/step - loss: 0.2601 - acc: 0.6590
Epoch 6/100
478/478 [==============================] - 0s 181us/step - loss: 0.2594 - acc: 0.6590
Epoch 7/100
478/478 [==============================] - 0s 159us/step - loss: 0.2588 - acc: 0.6590
Epoch 8/100
478/478 [==============================] - 0s 160us/step - loss: 0.2582 - acc: 0.6590
Epoch 9/100
478/478 [==============================] - 0s 170us/step - loss: 0.2576 - acc: 0.6590
Epoch 10/100
478/478 [==============================] - 0s 206us/step - loss: 0.2570 - acc: 0.6590
Epoch 11/100
478/47

478/478 [==============================] - 0s 152us/step - loss: 0.2136 - acc: 0.6590
Epoch 84/100
478/478 [==============================] - 0s 157us/step - loss: 0.2134 - acc: 0.6590
Epoch 85/100
478/478 [==============================] - 0s 300us/step - loss: 0.2132 - acc: 0.6590
Epoch 86/100
478/478 [==============================] - 0s 228us/step - loss: 0.2129 - acc: 0.6590
Epoch 87/100
478/478 [==============================] - 0s 148us/step - loss: 0.2125 - acc: 0.6590
Epoch 88/100
478/478 [==============================] - 0s 224us/step - loss: 0.2124 - acc: 0.6590
Epoch 89/100
478/478 [==============================] - 0s 246us/step - loss: 0.2123 - acc: 0.6590
Epoch 90/100
478/478 [==============================] - 0s 196us/step - loss: 0.2120 - acc: 0.6590
Epoch 91/100
478/478 [==============================] - 0s 177us/step - loss: 0.2117 - acc: 0.6590
Epoch 92/100
478/478 [==============================] - 0s 191us/step - loss: 0.2117 - acc: 0.6590
Epoch 93/100
478/478 [=

In [34]:
def convert_age(age):
    if age < 0:
        print('You still alive!')
    else:
        years = int(age)
        months_float = 12 * (age - years)
        months = int(months_float)
        days_float = 30 * (months_float - months)
        days = int(days_float)
        
        print('You lived for {} years, {} months and {} days'.format(years, months, days))

In [35]:
my_name = encode_name(preprocess_name('Vinicius Matheus'))
my_name = pad_sequences([my_name], maxlen=MAX_NAME_SIZE, padding='post')
my_name.shape

(1, 50)

In [38]:
res = model.predict(np.array(my_name))[0][0]
age_scaled = scaler.inverse_transform(res)[0][0]
convert_age(age_scaled)

You lived for 8 years, 11 months and 21 days
